In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.guillaumerozier.fr
Mail : guillaume.rozier@telecomnancy.net

README:s
This file contains script that generate France maps and GIFs. 
Single images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.
I'm currently cleaning this file, please ask me is something is not clear enough!
Requirements: please see the imports below (use pip3 to install them).

"""

"\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.guillaumerozier.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:s\nThis file contains script that generate France maps and GIFs. \nSingle images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.\nI'm currently cleaning this file, please ask me is something is not clear enough!\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [2]:
import france_data_management as data
import pandas as pd
from tqdm import tqdm
import json
import plotly.express as px
from datetime import datetime
import imageio
import multiprocessing
import locale
import shutil
import os
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [3]:
# Import data from Santé publique France
_, _, _, _, _, _, _, df_incid, _ = data.import_data()
with open('data/france/dep.geojson') as response:
    depa = json.load(response)

  0%|          | 0/4 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



        dep date_de_passage sursaud_cl_age_corona  nbre_pass_corona  \
0        01      2020-02-24                     0               0.0   
1        01      2020-02-24                     A               0.0   
2        01      2020-02-24                     B               0.0   
3        01      2020-02-24                     C               0.0   
4        01      2020-02-24                     D               0.0   
...     ...             ...                   ...               ...   
123013  976      2020-09-13                     A               0.0   
123014  976      2020-09-13                     B               0.0   
123015  976      2020-09-13                     C               0.0   
123016  976      2020-09-13                     D               0.0   
123017  976      2020-09-13                     E               0.0   

        nbre_pass_tot  nbre_hospit_corona  nbre_pass_corona_h  \
0               357.0                 0.0                 0.0   
1                

10it [00:09,  1.00it/s]                      


In [4]:
def build_map(data_df, img_folder, date_val, date_str = "date", dep_str = "departement", color_str = 'indic_synthese', legend_title="legend_title", title="title", subtitle=""):
    data_df = data_df[data_df[date_str] == date]

    fig = px.choropleth(geojson = depa, 
                        locations = data_df[dep_str], 
                        featureidkey="properties.code",
                        color = data_df[color_str],
                        scope='europe',
                        labels={color_str:"Couleur"},
                        #color_discrete_sequence = ["green", "orange", "red"],
                        #labels={'red':"Couleur", 'orange':'bla', 'green':'lol'},
                        color_discrete_map = {"Vert (<25)":"green", "Orange (25-50)":"orange", "Rouge (>50)":"red"}
                        #category_orders = {"indic_synthese" :["vert", "orange", "rouge"]}
                              )
    date_title = datetime.strptime(date_val, '%Y-%m-%d').strftime('%d %B')
    date_now = datetime.now().strftime('%d %B')
    
    fig.update_geos(fitbounds="locations", visible=False)

    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        title={
            'text': title,
            'y':0.98,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},

        titlefont = dict(
            size=30),

        annotations = [
            dict(
                x=0.54,
                y=0.03,
                xref='paper',
                yref='paper',
                xanchor = 'center',
                text='Source : Santé publique France. Auteur : @guillaumerozier.',
                showarrow = False
            ),

            dict(
                x=0.55,
                y=0.94,
                xref='paper',
                yref='paper',
                text= "{}<br>{} (données du {})".format(subtitle, date_now, date_title),
                showarrow = False,
                font=dict(
                    size=20
                        )
            )]
         ) 

    fig.update_geos(
        #center=dict(lon=-30, lat=-30),
        projection_rotation=dict(lon=12, lat=30, roll=8),
        #lataxis_range=[-50,20], lonaxis_range=[0, 200]
    )
    #fig.show()
    if date == dates_deconf[-1]:
        fig.write_image((img_folder+"/{}.jpeg").format("latest"), scale=2, width=1200, height=800)
    fig.write_image((img_folder+"/{}.jpeg").format(date), scale=2, width=1200, height=800)

In [5]:
def build_gif(file_gif, imgs_folder, dates):
    i=0
    with imageio.get_writer(file_gif, mode='I', duration=0.3) as writer: 
        for date in tqdm(dates):
            print((imgs_folder+"/{}.jpeg").format(date))
            image = imageio.imread((imgs_folder+"/{}.jpeg").format(date))
            writer.append_data(image)
            i+=1
            if i==len(dates):
                for k in range(8):
                    writer.append_data(image)

In [6]:
dates_deconf = list(dict.fromkeys(list(df_incid["jour"].values)))

date = dates_deconf[-1]
build_map(df_incid, "images/charts/france/dep-map-incid-cat", date_val=date, date_str = "jour", dep_str = "dep", color_str = 'incidence_color', legend_title="", title="Incidence", subtitle="Nombre de cas hebdomadaires pour 100 000 habitants")

In [7]:
build_gif("images/charts/france/incid-cat.gif", "images/charts/france/dep-map-incid-cat", dates_deconf[-30:-1])

  0%|          | 0/29 [00:00<?, ?it/s]

images/charts/france/dep-map-incid-cat/2020-08-13.jpeg


  3%|▎         | 1/29 [00:00<00:21,  1.27it/s]

images/charts/france/dep-map-incid-cat/2020-08-14.jpeg


  7%|▋         | 2/29 [00:01<00:20,  1.29it/s]

...

 93%|█████████▎| 27/29 [00:27<00:01,  1.04it/s]

images/charts/france/dep-map-incid-cat/2020-09-09.jpeg


 97%|█████████▋| 28/29 [00:27<00:00,  1.07it/s]

images/charts/france/dep-map-incid-cat/2020-09-10.jpeg


100%|██████████| 29/29 [00:35<00:00,  1.21s/it]
